In this notebook I analyze the computer (level 1) performance against humans. 

In [36]:
import numpy as np
import random as random
#using simpleguitk for display, is not needed for computer game
import simpleguitk as simplegui
import time
#new ones
import pandas as pd
from xgboost import XGBRegressor
import os
#skyjo game classes and functions
from skyjo_functions4 import *
#own functions for machine learning
from ml_functions2 import *

In [13]:
myPath='/home/tobias/ml-testing/games/skyjo'
list_input_files=[f for f in os.listdir(myPath) 
    if f.startswith('human_computer1_') and os.path.isfile(os.path.join(myPath, f))]
list_input_files.sort()
print(list_input_files)

['human_computer1_001.txt', 'human_computer1_002.txt', 'human_computer1_003.txt', 'human_computer1_004.txt', 'human_computer1_005.txt', 'human_computer1_006.txt', 'human_computer1_007.txt', 'human_computer1_008.txt', 'human_computer1_009.txt', 'human_computer1_010.txt', 'human_computer1_011.txt', 'human_computer1_012.txt', 'human_computer1_013.txt', 'human_computer1_014.txt', 'human_computer1_015.txt', 'human_computer1_016.txt', 'human_computer1_017.txt', 'human_computer1_018.txt', 'human_computer1_019.txt', 'human_computer1_020.txt', 'human_computer1_021.txt', 'human_computer1_022.txt', 'human_computer1_023.txt', 'human_computer1_024.txt', 'human_computer1_025.txt', 'human_computer1_026.txt', 'human_computer1_027.txt', 'human_computer1_028.txt', 'human_computer1_029.txt', 'human_computer1_030.txt', 'human_computer1_031.txt', 'human_computer1_032.txt', 'human_computer1_033.txt', 'human_computer1_034.txt', 'human_computer1_035.txt', 'human_computer1_036.txt', 'human_computer1_037.txt', 

In [74]:
res=np.zeros((7,len(list_input_files)))
deltahuman=0
deltacomputer=0
#list of results, currect ones need to 
listf=[]
for i in range(len(list_input_files)):
    file=np.loadtxt(list_input_files[i])   
    x=0
    #get previous also to check that it is doe snot include also old input
    if i>0:
        ofile=np.loadtxt(list_input_files[i-1])  
        #append all if equal besides score or beyound game 50
        if sum(np.round(ofile[0:49,0]))!=sum(np.round(file[0:49,0])) or i>50:
            listf.append(file)
        else:
            #otherwise only append the difference
            x=1
            subfile=file[:,ofile.shape[1]:file.shape[1]]
            #chnage round number in it
            subfile[48]=range(subfile.shape[1])
            listf.append(subfile)
            #could add rounds from end, but better not since not possible in reality
    else:
        listf.append(file)
    res[0,i]=i+1
    #number of turns is overall not set zero for new game from game 50 correct
    if x==0:
        res[4,i]=file.shape[1]
        #start player #does not change in new game also from game 50 correct
        res[5,i]=file[0,0]
    else:
        res[4,i]=subfile.shape[1]
        res[5,i]=subfile[0,0]
    #start player #does not change in new game also from game 50 correct
    res[5,i]=file[0,0]
    #last player
    res[6,i]=file[0,file.shape[1]-1]
    #always player 0, human first 
    if file[0,0]==0:
        res[1:3,i]=file[49:51,0]
    else:
        res[1,i]=file[50,0]
        res[2,i]=file[49,0]
    if res[1,i]<res[2,i]:
        #human winner mark
        res[3,i]=1
        #to calculate by how much a  human wins in average 
        deltahuman+=(res[2,i]-res[1,i])
    else:
        deltacomputer+=(res[1,i]-res[2,i])

#print array
#the longest length of 78 was when i tested the vanihsing cards on purpose
print(res.T)        
#print number of elements in list
print(len(listf))

[[  1.  24.  46.   1.  30.   0.   1.]
 [  2.  36.   9.   0.  32.   0.   1.]
 [  3.  78.  24.   0.  31.   1.   1.]
 [  4.  11.  49.   1.  24.   0.   1.]
 [  5.   9.  22.   1.  26.   0.   1.]
 [  6.  13.  28.   1.  78.   1.   1.]
 [  7.   9.  19.   1.  41.   1.   1.]
 [  8.  10.  30.   1.  33.   1.   1.]
 [  9.  32.   8.   0.  31.   1.   1.]
 [ 10.  20.  24.   1.  41.   1.   1.]
 [ 11.  22.  34.   1.  34.   0.   1.]
 [ 12.  17.  41.   1.  34.   0.   1.]
 [ 13.  27.  29.   1.  27.   0.   1.]
 [ 14.  46.  15.   0.  31.   0.   1.]
 [ 15.  11.  23.   1.  32.   0.   1.]
 [ 16.  12.  36.   1.  24.   0.   1.]
 [ 17.  16.  22.   1.  35.   1.   1.]
 [ 18.  36.   4.   0.  36.   1.   1.]
 [ 19.  19.  28.   1.  24.   1.   1.]
 [ 20.  18.  32.   1.  36.   1.   1.]
 [ 21.  26.  13.   0.  36.   1.   1.]
 [ 22.  10.  55.   1.  27.   1.   1.]
 [ 23.  28.  41.   1.  36.   1.   1.]
 [ 24.  20.  65.   1.  24.   1.   1.]
 [ 25.  29.  39.   1.  26.   1.   1.]
 [ 26.  35.  45.   1.  28.   1.   1.]
 [ 27.  24. 

In [69]:
#convert lists of differnet lengths consistent of different arrays of same shape[0] but different shape[1]
def lists_arrays_to_one(listf,int2=True):
    #determine length of output
    c=0
    for i in range(len(listf)):
        c+=listf[i].shape[1]
        #create array
    ar=np.zeros((listf[0].shape[0],c))   
    c=0
    #now fill it
    for i in range(len(listf)):
        if int2==True:
            ar[:,c:c+listf[i].shape[1]]=np.round(listf[i])
        else:
            ar[:,c:c+listf[i].shape[1]]=np.round(listf[i])
        c+=listf[i].shape[1]
    return ar     

In [71]:
ar=lists_arrays_to_one(listf)
print(ar.shape)

(51, 2205)


In [8]:
print(f"the human won {int(sum(res[3]))} times of {res.shape[1]} rounds")

print(f"the human average is {round(np.mean(res[1]),3)} points")
                       
print(f"the computer average is {round(np.mean(res[2]),3)} points")
print(f"the human average adventage is  {round(np.mean(res[2]-res[1]),3)} points")
print(f"the human wins to {round(np.mean(res[3]*100),1)} %")

the human won 56 times of 69 rounds
the human average is 24.159 points
the computer average is 33.13 points
the human average adventage is  8.971 points
the human wins to 81.2 %


In [9]:
#average win is by to be determine deltapoints
print(f"human average adventage {round(deltahuman/sum(res[3]),3)} points")
print(f"computer average adventage {round(deltacomputer/(res.shape[1]-sum(res[3])),3)} points")

human average adventage 18.464 points
computer average adventage 31.923 points


(51, 145)
